<a href="https://colab.research.google.com/github/ReginaGH/AB_testing/blob/main/Homework_8_ab_test_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ_8
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

### Загрузка библиотек

In [2]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

from google.colab import drive

### Загрузка и первичный анализ данных

In [3]:
drive.mount('/content/drive')
data = pd.read_excel('/content/drive/MyDrive/Learning/gb_sem_8_hm.xlsx')
data.head()

Mounted at /content/drive


/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [4]:
data.info() # нулевых значений нет, форматы проверены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
# Посмотрим разделение выборок по группам. Можно сказать, группы равны по размеру.
data.groupby('VARIANT_NAME').count()

,USER_ID,REVENUE
VARIANT_NAME,,
control,4984,4984
variant,5016,5016


### Проверям, подготавливаем данные

In [6]:
data.USER_ID.unique().shape
# значит остальные (10_000 - 6_324) не уникальны

(6324,)

In [7]:
# удалим дубликаты в столбце USED_ID :
data_id_unique = data.drop_duplicates(subset='USER_ID')
data_id_unique.shape

(6324, 3)

In [8]:
unique_ids = data_id_unique.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME':'count'}).query('VARIANT_NAME == 1').USER_ID.values
df_new = data_id_unique[data_id_unique.USER_ID.isin(unique_ids)].copy(deep=True)
df_new.groupby('VARIANT_NAME')['REVENUE'].describe()

,count,mean,std,min,25%,50%,75%,max
VARIANT_NAME,,,,,,,,
control,3132.0,0.126986,3.584689,0.0,0.0,0.0,0.0,196.01
variant,3192.0,0.072694,1.256534,0.0,0.0,0.0,0.0,57.46


### Анализ данных

In [9]:
# Разделим на тестовую и контрольную группы:
control = df_new[df_new['VARIANT_NAME']=='control'].copy(deep=True)
test = df_new[df_new['VARIANT_NAME']=='variant'].copy(deep=True)

In [10]:
def continious_result(control: pd.DataFrame,
                      test: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    test_mean = test.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    test_std = test.loc[:, column].std(ddof=1)

    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        test_sample = test.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - test_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=test_mean, sd1=test_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [13]:
continious_result(control, test, column='REVENUE')

100%|██████████| 10000/10000 [00:04<00:00, 2046.44it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.020211,0.42532,0.874143,"[-0.049, 0.209]",0.054414


### Визуальный анализ данных

In [14]:
import plotly.express as plex
fig = plex.histogram(data,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='Revenue by user',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

Вывод по графику: экспонциальное распределение (большинство покупок = 0)

### Вывод: 
Полученный результат не является статистически значимым по трём параметрам:
- alpha и beta значительно превышают норму,
- доверительный интервал включает "0".
